# **Chapter 2: Chain of Thought Prompting**

---
**Lesson:**

If someone woke you up and immediately started asking you several complicated questions that you had to respond to right away, how would you do? Probably not as good as if you were given time to think step by step through your answer first. Just like a human, Meta Llama2 benefits significantly from a structured approach to problem-solving. Allowing Meta the opportunity to process information in a sequential, step-by-step manner enhances its precision and effectiveness, especially in tackling intricate tasks. This methodical approach, known as 'chain of thought (CoT),' is important when extracting the most accurate and insightful responses from Llama2 in complex scenarios.

For each of these prompts, the CoT approach helps break down complex topics into more manageable parts, guiding the model through a logical progression of thoughts and considerations. This not only makes the model's outputs more understandable but also can improve the accuracy and depth of the responses.

---

First we will setup our dependencies

In [2]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.304

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

from langchain.llms.bedrock import Bedrock
from langchain import PromptTemplate

modelId = 'meta.llama2-13b-chat-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

def invoke_model_and_get_response(prompt_data): 
    body = json.dumps({ 
        'prompt': prompt_data,
        'max_gen_len': 1024,
        'top_p': 0.9,
        'temperature': 0.2
    })

    try:
        response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        response_body = json.loads(response.get('body').read().decode('utf-8'))
        outputText = response_body['generation'].strip()
        return outputText

    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'AccessDeniedException':
            return (f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubleshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        else:
            raise

# **Examples:**

**Example 2.1 - History Lesson**

In [3]:
# create the prompt
prompt_data = """
[INST] Describe the causes of World War I.
Begin by identifying the main political and economic tensions in Europe before 1914.
Then, discuss the role of alliances and militarism in escalating these tensions.
Conclude by explaining how the assassination of Archduke Franz Ferdinand was the immediate trigger for the war.
[\INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

Sure, I'd be happy to help! Here's my answer:

Before 1914, Europe was a complex web of political and economic tensions. The main powers of the continent, including Germany, France, Britain, Russia, and Austria-Hungary, had been engaged in a long-standing struggle for dominance and security.

One of the key issues was the question of imperialism and colonies. European powers had been competing for territories and resources around the world for centuries, and this competition had created tensions between them. Germany, in particular, had been rapidly expanding its colonial empire in Africa and the Pacific, which had raised concerns among the other European powers.

Another major issue was the complex system of alliances that had developed among the European powers. These alliances were formed for mutual defense and security, but they also created a situation in which a small conflict between two countries could quickly escalate into a larger, global war.

Militarism was also a major fac

**Example 2.2 - Water Cycle**

In [4]:
# create the prompt
prompt_data = """
[INST] Explain the water cycle and its importance to Earth's climate system
Start by describing how water evaporates from the Earth's surface. 
Next, explain the process of condensation in the atmosphere. 
Then, outline how precipitation returns water to the surface. 
Finally, discuss the role of the water cycle in regulating temperature and supporting life
[\INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

Sure, I'd be happy to explain the water cycle and its importance to Earth's climate system!

The water cycle, also known as the hydrologic cycle, is the continuous process by which water is circulated between the Earth's oceans, atmosphere, and land. The cycle is driven by the sun's energy and involves the movement of water in three phases: liquid, solid (ice), and gas (water vapor).

First, let's start with the process of evaporation, which occurs when the sun heats water bodies such as oceans, lakes, and rivers, causing the water to turn into water vapor. This process also occurs through transpiration, where plants release water vapor into the air. As the water vapor rises into the atmosphere, it cools and condenses into tiny droplets, forming clouds.

Next, the process of condensation occurs when the water vapor in the clouds becomes saturated with water and begins to condense into larger droplets. This process releases heat, which helps to drive the water cycle.

Finally, precipita

**Example 2.3 - AI Ethics**	

In [5]:
prompt_data = """
[INST]
Discuss the ethical implications of artificial intelligence in decision-making processes.
First, outline the benefits of using AI for making complex decisions. 
Then, explore potential risks, such as bias and lack of accountability. 
Next, consider the ethical principles that should guide the integration of AI in decision-making. 
Conclude by proposing measures to mitigate risks while maximizing benefits.
[\INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

Sure, I'd be happy to help with that! Here's my outline:

I. Benefits of using AI for making complex decisions

* Improved accuracy and speed
* Ability to process large amounts of data
* Enhanced decision-making consistency
* Reduced potential for human bias

II. Potential risks of using AI for making complex decisions

* Bias in training data
* Lack of accountability and transparency
* Potential for discrimination
* Unintended consequences

III. Ethical principles for integrating AI in decision-making

* Transparency and explainability
* Accountability and responsibility
* Fairness and non-discrimination
* Human oversight and intervention

IV. Measures to mitigate risks and maximize benefits

* Regular auditing and testing of AI systems
* Diverse and representative training data
* Human-centered design and development
* Continuous monitoring and evaluation

Now, let me expand on each of these points:

I. Benefits of using AI for making complex decisions

Artificial intelligence has th

# **Exercises**

The following exercises will need you to manipulate the prompt to get the desired output

**Exercise 2.1 - Classifying Emails**

In this exercise, we'll be instructing Llama2 to sort emails into the following categories: (A) Pre-sale question (B) Broken or defective item (C) Billing question (D) Other (please explain)

For this exercise, change the prompt in the cell below to make Llama2 output the correct classification. Your answer needs to include the letter (A - D) of the correct choice, with the parentheses, as well as the name of the category.

In [6]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], #Change Input variables 
    template="""
[INST]
Please classify this email: 

<email>
{EMAIL} 
</email>

Here are the 4 categories it can be classified into, you must choose one:
(A) Pre-sale question
(B) Broken or defective item
(C) Billing question 
(D) Other (please explain)
[/INST]
"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.") #Should be classified as (B)

response = invoke_model_and_get_response(prompt)
print(response)

Sure! Based on the content of the email, I would classify it as (B) Broken or defective item. The user is reporting a strange noise and smoky smell coming from their Mixmaster4000, which suggests that it may be malfunctioning or damaged.


# Chapter 2 - END.